<a href="https://colab.research.google.com/github/wombat-42/AI-LAB/blob/main/WEEK_9_AND_WEEK_10_alpha_beta_pruning_AND_resolution_1BM22CS053.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

# Helper functions to handle literals and predicates
def is_variable(term):
    """Check if a term is a variable (lowercase without parentheses)."""
    return term.islower() and "(" not in term and ")" not in term

def split_literal(literal):
    """Helper function to handle splitting of literal into predicate and arguments."""
    if "(" in literal:
        pred, args = literal.split("(")
        args = args[:-1]  # Remove closing ')'
        args = args.split(",")
        return pred, args
    else:
        return literal, []

def unify_literal(lit1, lit2):
    """Unify two literals if possible, returning a substitution map or None."""
    if lit1 == lit2:
        return {}

    # Handle negation: If one literal is negated, remove the negation for comparison
    if lit1.startswith("¬"):
        lit1 = lit1[1:]
    if lit2.startswith("¬"):
        lit2 = lit2[1:]

    # Now split the literals into predicate and arguments
    pred1, args1 = split_literal(lit1)
    pred2, args2 = split_literal(lit2)

    if pred1 != pred2:
        return None

    subst = {}
    for a1, a2 in zip(args1, args2):
        if a1 != a2:
            if is_variable(a1):
                subst[a1] = a2
            elif is_variable(a2):
                subst[a2] = a1
            else:
                return None
    return subst

def apply_substitution(clause, substitution):
    """Apply a substitution to a clause."""
    new_literals = []
    for lit in clause.literals:
        new_lit = lit
        for var, const in substitution.items():
            new_lit = new_lit.replace(var, const)
        new_literals.append(new_lit)
    return Clause(new_literals)

class Clause:
    def __init__(self, literals):
        self.literals = set(literals)

    def __repr__(self):
        return " ∨ ".join(sorted(self.literals))

    def resolve(self, other):
        """Perform resolution between two clauses."""
        resolvents = []
        for literal in self.literals:
            for other_literal in other.literals:
                if literal.startswith("¬"):
                    neg_literal = literal[1:]
                else:
                    neg_literal = "¬" + literal

                unifier = unify_literal(literal, neg_literal)
                if unifier is not None:
                    new_clause = Clause((self.literals - {literal}) | (other.literals - {other_literal}))
                    resolvents.append(apply_substitution(new_clause, unifier))
        return resolvents

def convert_to_cnf(formula):
    """Convert a propositional formula into CNF."""
    # Step 1: Handle Implication (P → Q is equivalent to ¬P ∨ Q)
    formula = formula.replace("→", "or")  # P → Q becomes ¬P ∨ Q

    # Step 2: Handle Biconditional (P ↔ Q is equivalent to (P → Q) ∧ (Q → P))
    formula = formula.replace("↔", "and")  # P ↔ Q becomes (P → Q) ∧ (Q → P)

    # Step 3: Remove extra spaces and handle 'not' for negation explicitly
    formula = formula.replace("not", "¬")  # Standardize 'not' to '¬' for negation

    # Step 4: Distribute ANDs over ORs to achieve CNF
    # Example: (P ∧ Q) ∨ R → (P ∨ R) ∧ (Q ∨ R)

    # This is a simple formula conversion; a more complex algorithm is needed for advanced cases
    return formula

def parse_to_clauses(cnf_formula):
    """Convert the CNF formula into Clause objects."""
    cnf_clauses = []
    # Split formula by conjunctions (and)
    for clause in cnf_formula.split("and"):
        literals = set(clause.split("or"))  # Split literals by disjunctions (or)
        cnf_clauses.append(Clause([lit.strip() for lit in literals]))
    return cnf_clauses

def PL_RESOLUTION(KB, query):
    """Performs propositional logic resolution."""
    # Negate the query and add it to the knowledge base
    negated_query = Clause([f"¬{query}"])
    clauses = KB + [negated_query]

    new = set()
    seen_pairs = set()

    while True:
        pairs = [(clauses[i], clauses[j]) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]

        for ci, cj in pairs:
            if (ci, cj) in seen_pairs or (cj, ci) in seen_pairs:
                continue
            seen_pairs.add((ci, cj))

            resolvents = ci.resolve(cj)
            for resolvent in resolvents:
                if not resolvent.literals:
                    return True  # Found the empty clause, meaning a contradiction was derived
                new.add(frozenset(resolvent.literals))

        # If no new clauses were generated, return False (no contradiction can be derived)
        if new.issubset(set(map(frozenset, (c.literals for c in clauses)))):  # Check if new clauses are subsets
            return False

        # Add the new clauses to the set of clauses
        for literals in new - set(map(frozenset, (c.literals for c in clauses))):
            clauses.append(Clause(list(literals)))

        new.clear()

# Function to allow user input
def get_user_input():
    """Function to receive user input for propositional logic formulas and queries."""
    user_formulas = []
    print("Enter your propositional logic formulas (enter 'done' when finished):")

    while True:
        formula = input("Enter formula: ")
        if formula.lower() == "done":
            break
        user_formulas.append(formula)

    queries = []
    print("\nEnter queries to prove (enter 'done' when finished):")
    while True:
        query = input("Enter query: ")
        if query.lower() == "done":
            break
        queries.append(query)

    return user_formulas, queries

# Main code execution
if __name__ == "__main__":
    # Get user input formulas and queries
    user_formulas, queries = get_user_input()

    # Convert each formula to CNF and parse them
    KB = []
    for formula in user_formulas:
        print(f"\nOriginal Formula: {formula}")

        # Convert to CNF
        cnf_formula = convert_to_cnf(formula)
        print(f"Converted to CNF: {cnf_formula}")

        # Parse CNF into clauses and add to KB
        KB.extend(parse_to_clauses(cnf_formula))

    # Resolve each query
    for query in queries:
        print(f"\nProving query: {query}")
        if PL_RESOLUTION(KB, query):
            print(f"The conclusion '{query}' is proven by resolution.")
        else:
            print(f"The conclusion '{query}' cannot be proven.")


Enter your propositional logic formulas (enter 'done' when finished):
Enter formula: Mother(leela,Oshin)
Enter formula: Alive(leela)
Enter formula: Mother(x,y) -> parent(x,y)
Enter formula: (parent(x,y) and Alive(x) ) -> Older(x,y)
Enter formula: done

Enter queries to prove (enter 'done' when finished):
Enter query: Older(leela, Oshin)
Enter query: done

Original Formula: Mother(leela,Oshin)
Converted to CNF: Mother(leela,Oshin)

Original Formula: Alive(leela)
Converted to CNF: Alive(leela)

Original Formula: Mother(x,y) -> parent(x,y)
Converted to CNF: Mother(x,y) -> parent(x,y)

Original Formula: (parent(x,y) and Alive(x) ) -> Older(x,y)
Converted to CNF: (parent(x,y) and Alive(x) ) -> Older(x,y)

Proving query: Older(leela, Oshin)
The conclusion 'Older(leela, Oshin)' is proven by resolution.


In [3]:

MAX, MIN = 99999, -99999


def minimax(depth, nodeIndex, maximizingPlayer,
			values, alpha, beta):


	if depth == 3:
		return values[nodeIndex]

	if maximizingPlayer:

		best = MIN

		# Recur for left and right children
		for i in range(0, 2):

			val = minimax(depth + 1, nodeIndex * 2 + i,
						False, values, alpha, beta)
			best = max(best, val)
			alpha = max(alpha, best)

			# Alpha Beta Pruning
			if beta <= alpha:
				break

		return best

	else:
		best = MAX

		# Recur for left and
		# right children
		for i in range(0, 2):

			val = minimax(depth + 1, nodeIndex * 2 + i,
							True, values, alpha, beta)
			best = min(best, val)
			beta = min(beta, best)

			# Alpha Beta Pruning
			if beta <= alpha:
				break

		return best


if __name__ == "__main__":

	values = [10,9,14,18,5,4,50,3]
	print(values)
	print("The optimal value is :", minimax(0, 0, True, values, MIN, MAX))



[10, 9, 14, 18, 5, 4, 50, 3]
The optimal value is : 10
